In [3]:
# %%
!pip install -q h5py typing-extensions wheel
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q datasets

# %%
!nvidia-smi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 2.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.1 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 

In [4]:
# %% [markdown]
# 
# ## Load Pre-trained model and tokenizer

# %%
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
model_id = "Qwen/Qwen2.5-3B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True, # Activate nested quantization for 4-bit base models (double quantization)
    bnb_4bit_quant_type="nf4", # Quantization type (fp4 or nf4), According to QLoRA paper, for training 4-bit base models (e.g. using LoRA adapters) one should use
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
# model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)


# %% [markdown]
# ## Preprocess the quantized model for training

# %%
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# %%
from peft import LoraConfig, get_peft_model

# You can try differnt parameter-effient strategy for model trianing, for more info, please check https://github.com/huggingface/peft
config = LoraConfig(
    r=8,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print("cell finished")

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

2025-04-16 06:02:24.707135: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744783345.163035      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744783345.290774      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

cell finished


In [5]:
# %% [markdown]
# ## Chat Template Usage

# %%
from jinja2 import Template
template = Template(tokenizer.chat_template)
message = "Please introduce yourself"
print(f"message:\n{message}\n")
message_send_to_model=template.render(messages=[{"role": "user", "content": message}],bos_token=tokenizer.bos_token,add_generation_prompt=True)
print(f"message_send_to_model:\n{message_send_to_model}")

# %%
template = Template(tokenizer.chat_template)
@torch.no_grad()
def generate(prompt):
    modelInput=template.render(messages=[{"role": "user", "content": prompt}],bos_token= tokenizer.bos_token,add_generation_prompt=True)
    print("-"*80)
    print(f"model_input_string:\n{modelInput}")
    input_ids = tokenizer.encode(modelInput, add_special_tokens=False, return_tensors='pt').to("cuda:0")
    outputs = model.generate(input_ids, do_sample=False)
    model_return_string = tokenizer.decode(*outputs, skip_special_tokens=False)
    print("-"*80)
    print(f"model_return_string:\n{model_return_string}")
    generated_ids = outputs[:, input_ids.shape[1]:]
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
    return generated_text

query = "Please introduce yourself"
print("-"*80)
print(f"query:\n{query}")
response = generate(query)
print("-"*80)
print(f"response:\n{response}")

# %% [markdown]
# ## Data Preparation

# %% [markdown]
# Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos toke

message:
Please introduce yourself

message_send_to_model:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Please introduce yourself<|im_end|>
<|im_start|>assistant

--------------------------------------------------------------------------------
query:
Please introduce yourself
--------------------------------------------------------------------------------
model_input_string:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Please introduce yourself<|im_end|>
<|im_start|>assistant

--------------------------------------------------------------------------------
model_return_string:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Please introduce yourself<|im_end|>
<|im_start|>assistant
Of course! I'm Qwen, a large language model created by Alibaba Cloud. My primary function
-------------

In [43]:
# original huatuo dataset
# # %%
# from datasets import load_dataset

# data = load_dataset("Abirate/english_quotes")

# dataset = load_dataset("FreedomIntelligence/Huatuo26M-Lite")
# dataset = dataset['train'].map(lambda sample: {"conversations": [{"from": "human", "value": sample['question']}, {"from": "gpt", "value": sample['answer']}]}, batched=False)

# dataset[3]

{'id': 21833024,
 'answer': '脓包型痘痘是由于毛囊皮脂腺分泌过多油脂，导致毛囊口被堵塞，细菌感染后形成的。治疗时需要内调外治，从身体内部调理，同时采用外用药物进行治疗。外用药物要选择无激素无依赖性的药物，避免对皮肤造成伤害。此外，洗脸时要使用温水，选用温和的洗面奶，保护皮肤基层。注意休息，避免熬夜，远离辐射性的电子产品。饮食上要避免辛辣刺激性食物，多吃蔬菜水果，保持肠胃健康。',
 'score': 4,
 'label': '皮肤性病科',
 'question': '我脸上总是长脓包型痘痘，经常复发。刚长的时候，脓包是硬硬的，还很疼，时间长了就会有脓头。脓包型痘痘怎么治疗？',
 'related_diseases': '',
 'conversations': [{'from': 'human',
   'value': '我脸上总是长脓包型痘痘，经常复发。刚长的时候，脓包是硬硬的，还很疼，时间长了就会有脓头。脓包型痘痘怎么治疗？'},
  {'from': 'gpt',
   'value': '脓包型痘痘是由于毛囊皮脂腺分泌过多油脂，导致毛囊口被堵塞，细菌感染后形成的。治疗时需要内调外治，从身体内部调理，同时采用外用药物进行治疗。外用药物要选择无激素无依赖性的药物，避免对皮肤造成伤害。此外，洗脸时要使用温水，选用温和的洗面奶，保护皮肤基层。注意休息，避免熬夜，远离辐射性的电子产品。饮食上要避免辛辣刺激性食物，多吃蔬菜水果，保持肠胃健康。'}]}

In [6]:
# load 法律数据集
from datasets import load_dataset, Features, Value, Sequence, concatenate_datasets

# 定义column转换方法
def convert_to_conversation(sample):
    return {
        "conversations": [
            {"from": "human", "value":  sample['input']},
            {"from": "gpt", "value": sample['output']}
        ]
    }

dataset = load_dataset("ShengbinYue/DISC-Law-SFT", data_files=['DISC-Law-SFT-Pair-QA-released.jsonl', 'DISC-Law-SFT-Pair.jsonl'])

# 应用转换
dataset = dataset['train'].map(
    convert_to_conversation,
    remove_columns=dataset['train'].column_names,  # 移除原始列
    batched=False
)

features = Features({
    'id': Value('string'),
    'reference': Sequence(Value('string')),
    'input': Value('string'),
    'output': Value('string')
})

dataset2 = load_dataset("ShengbinYue/DISC-Law-SFT", features = features,data_files=['DISC-Law-SFT-Triplet-QA-released.jsonl', 'DISC-Law-SFT-Triplet-released.jsonl'])

dataset2 = dataset2['train'].map(
    convert_to_conversation,
    remove_columns=dataset2['train'].column_names,  # 移除原始列
    batched=False
)

merged_dataset = concatenate_datasets([dataset, dataset2])

README.md:   0%|          | 0.00/3.65k [00:00<?, ?B/s]

DISC-Law-SFT-Pair-QA-released.jsonl:   0%|          | 0.00/94.0M [00:00<?, ?B/s]

DISC-Law-SFT-Pair.jsonl:   0%|          | 0.00/347M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/246450 [00:00<?, ? examples/s]

DISC-Law-SFT-Triplet-QA-released.jsonl:   0%|          | 0.00/85.5M [00:00<?, ?B/s]

DISC-Law-SFT-Triplet-released.jsonl:   0%|          | 0.00/52.8M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/39331 [00:00<?, ? examples/s]

In [7]:
merged_dataset

Dataset({
    features: ['conversations'],
    num_rows: 285781
})

In [10]:
# %%
from torch.utils.data import random_split
train_dataset_size, val_dataset_size = 40, 8
# train_dataset, val_dataset, _ = random_split(dataset, [train_dataset_size, val_dataset_size, len(dataset)-train_dataset_size-val_dataset_size])
train_dataset, val_dataset, _ = random_split(dataset2, [0.4,0.4,0.2])
print(train_dataset[0]['conversations'])

# %% [markdown]
# ### Customized Dataset
# Create a specialized dataset class named "InstructionDataset" designed to handle our custom dataset.

# %%
import transformers
from typing import Dict, Sequence, List
from torch.utils.data import Dataset
from dataclasses import dataclass

def preprocess(
    sources,
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    template = Template(tokenizer.chat_template)
    max_seq_len = tokenizer.model_max_length
    messages = []
    for i, source in enumerate(sources):
        if source[0]["from"] != "human":
            # Skip the first one if it is not from human
            source = source[1:]

        for j in range(0, len(source), 2):
            if j+1 >= len(source): continue
            q = source[j]["value"]
            a = source[j+1]["value"]
            assert q is not None and a is not None, f'q:{q} a:{a}'
            input =  template.render(messages=[{"role": "user", "content": q},{"role": "assistant", "content": a}],bos_token=tokenizer.bos_token,add_generation_prompt=False)
            input_ids = tokenizer.encode(input, add_special_tokens= False)

            query = template.render(messages=[{"role": "user", "content": q}],bos_token=tokenizer.bos_token,add_generation_prompt=True)
            query_ids = tokenizer.encode(query, add_special_tokens= False)

            labels = [-100]*len(query_ids) + input_ids[len(query_ids):]
            assert len(labels) == len(input_ids)
            if len(input_ids) == 0: continue
            messages.append({"input_ids": input_ids[-max_seq_len:], "labels": labels[-max_seq_len:]})

    input_ids = [item["input_ids"] for item in messages]
    labels = [item["labels"] for item in messages]

    max_len = max(len(x) for x in input_ids)

    max_len = min(max_len, max_seq_len)
    input_ids = [ item[:max_len] + [tokenizer.eos_token_id]*(max_len-len(item)) for item in input_ids]
    labels = [ item[:max_len] + [-100]*(max_len-len(item)) for item in labels]

    input_ids = torch.LongTensor(input_ids)
    labels = torch.LongTensor(labels)
    return {
        "input_ids": input_ids,
        "labels": labels
    }


class InstructDataset(Dataset):
    def __init__(self, data: Sequence, tokenizer: transformers.PreTrainedTokenizer) -> None:
        super().__init__()
        self.tokenizer = tokenizer
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index) -> Dict[str, torch.Tensor]:
        sources = self.data[index]
        if isinstance(index, int):
            sources = [sources]
        data_dict = preprocess([e['conversations'] for e in sources], self.tokenizer)
        if isinstance(index, int):
            data_dict = dict(input_ids=data_dict["input_ids"][0], labels=data_dict["labels"][0])
        return data_dict


@dataclass
class DataCollatorForSupervisedDataset(object):
    tokenizer: transformers.PreTrainedTokenizer
    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids,
            batch_first=True,
            padding_value=self.tokenizer.pad_token_id)
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

# %%
train_dataset = InstructDataset(train_dataset, tokenizer)
val_dataset = InstructDataset(val_dataset, tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
print("cell finished")

[{'from': 'human', 'value': '《民法典》第一百八十七条：民事主体因同一行为应当承担民事责任、行政责任和刑事责任的，承担行政责任或者刑事责任不影响承担民事责任；民事主体的财产不足以支付的，优先用于承担民事责任。\n《民法典》第一百八十八条：向人民法院请求保护民事权利的诉讼时效期间为三年。法律另有规定的，依照其规定。诉讼时效期间自权利人知道或者应当知道权利受到损害以及义务人之日起计算。法律另有规定的，依照其规定。但是，自权利受到损害之日起超过二十年的，人民法院不予保护，有特殊情况的，人民法院可以根据权利人的申请决定延长。\n《民法典》第一百八十五条：侵害英雄烈士等的姓名、肖像、名誉、荣誉，损害社会公共利益的，应当承担民事责任。\n《民法典》第一百八十六条：因当事人一方的违约行为，损害对方人身权益、财产权益的，受损害方有权选择请求其承担违约责任或者侵权责任。\n<问题>：\n小明购买了一部英雄烈士的肖像画作装饰他的家。但是，他的邻居认为这是对英雄烈士荣誉的侵害，向小明提出了民事诉讼。根据相关法规，该案件的承担责任是怎样分配的？'}, {'from': 'gpt', 'value': '根据所提供的法律条文及相关法规，结合事实，对于该案件的承担责任的分配如下：\n\n根据《民法典》第一百八十五条，侵害英雄烈士等的姓名、肖像、名誉、荣誉，损害社会公共利益的行为，应当承担民事责任。\n\n根据《民法典》第一百八十六条，应受损害方有权选择请求对方承担违约责任或者侵权责任。\n\n综上所述，根据相关法律规定，如果小明的购买行为被视为侵害英雄烈士荣誉，他可能需要承担民事责任，应受邻居提起的民事诉讼。然而，最终责任的确定要取决于法庭对案件的审理，并综合考虑具体的事实和证据。'}]
cell finished


In [11]:
!nvidia-smi

Wed Apr 16 06:21:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P0             27W /   70W |    3445MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
# %%
sample_data = train_dataset[9]
IGNORE_INDEX=-100

print("=" * 80)
print("Debuging: ")
print(f"Input_ids\n{sample_data['input_ids']}")
print(f"Label_ids\n{sample_data['labels']}")
print("-" * 80)
print(f"Input:\n{tokenizer.decode(sample_data['input_ids'])}")
print("-" * 80)
N_id = tokenizer.encode("N", add_special_tokens= False)[0]
print(f"Label:\n{tokenizer.decode([N_id if x == -100 else x for x in sample_data['labels']])}")
print("=" * 80)


# %% [markdown]
# ## Training

# %% [markdown]
# ### General Training Hyperparameters

# %%
# Set training parameters
training_arguments = transformers.TrainingArguments(
    output_dir="./checkpoints",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    optim='paged_adamw_32bit',
    save_steps=0,
    logging_steps=1,
    learning_rate=2e-7,
    weight_decay=0.001,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    gradient_checkpointing=True,
    report_to="none",
    dataloader_drop_last=True
)

# %%

model.train()
trainer = transformers.Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator
)
trainer.train()

Debuging: 
Input_ids
tensor([151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
           553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
            13, 151645,    198, 151644,    872,    198, 100345, 101995,  27369,
          3837, 104538, 107976, 101995,   9370, 103941,  99985,  99351,   8997,
         55135,  99463,  22697,  99532, 103424, 114006,   3837,     17,     15,
            16,     17, 108798,     17,     15,     16,     18,   7948,   3837,
        108360, 100453,  99569, 104828, 101962,  39352, 102648,   3837, 104692,
        109054,  55135,  99463,  99523, 104654,  45629,  99277,  99605, 102925,
          9370, 109728,   3837, 100751, 117977, 102452,  99385,   3837,  54899,
        109054, 109728, 100210,  93488,     19,     15,  88683, 102131,   1773,
         53393, 109494,  69177,  22697, 115101,  89977, 105379,   5122,  55135,
         99463,  22697,  55135,  99463,  99523, 101973, 100453,  99569,  20412,
         67338, 109

/tmp/ipykernel_31/3876896825.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


KeyboardInterrupt: 

In [7]:
# %%
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

model.print_trainable_parameters()

# %% [markdown]
# Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

# %%
import math
!pip install -q -U git+https://github.com/huggingface/accelerate.git
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

# %% [markdown]
# ## Save Trained LoRA

trainable params: 1,261,568 || all params: 7,616,878,080 || trainable%: 0.0166


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Perplexity: 14.66


In [8]:
# %%
!pwd
output_path = "ilora"
trainer.save_model(output_path)

# %% [markdown]
# ### Test the trained model

# %%
template = Template(tokenizer.chat_template)
@torch.no_grad()
def generate(prompt):
    modelInput = template.render(messages=[{"role": "user", "content": prompt}],bos_token= tokenizer.bos_token,add_generation_prompt=True)
    input_ids = tokenizer.encode(modelInput, add_special_tokens=False, return_tensors='pt').to("cuda:0")
    outputs = model.generate(input_ids, temperature=1.0)
    model_return_string = tokenizer.decode(*outputs, skip_special_tokens=False)
    print("-"*80)
    print(f"model_return_string:\n{model_return_string}")
    generated_ids = outputs[:, input_ids.shape[1]:]
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
    return generated_text

query = "I get hit"
print(f"query:\n{query}")
response = generate(query)
print("-"*80)
print(f"response:\n{response}")

# %% [markdown]
# # Clean GPU Memory

# %%
!nvidia-smi

/kaggle/working


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


query:
I get hit
--------------------------------------------------------------------------------
model_return_string:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
I get hit<|im_end|>
<|im_start|>assistant
I'm sorry to hear that you're feeling hurt. Can you provide more context about what happened?
--------------------------------------------------------------------------------
response:
I'm sorry to hear that you're feeling hurt. Can you provide more context about what happened?
Sat Apr 12 21:02:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |  

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [33]:
# %%
# Empty VRAM
# del model
# del trainer
import gc
import torch
torch.cuda.empty_cache()
gc.collect()
gc.collect()

# %%
!nvidia-smi

Sat Apr 12 21:20:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P0             32W /  250W |    7581MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
# %% [markdown]
# ## Load the trained model back and integrate the trained LoRA within.

# %%
from peft import PeftModel



model = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map={"":0})
model = PeftModel.from_pretrained(model, output_path)
model = model.merge_and_unload()
model.config.max_length = 512
model.eval()

tokenizer = transformers.AutoTokenizer.from_pretrained(model_id, padding_side="left")
# tokenizer.pad_token = tokenizer.unk_token


# %% [markdown]
# ## Answer generation

# %%
@torch.no_grad()
def generate(prompts):
    model_inputs = [template.render(messages=[{"role": "user", "content": prompt}], bos_token=tokenizer.bos_token, add_generation_prompt=True) for prompt in prompts]
    input_ids = tokenizer(model_inputs, add_special_tokens=False, return_tensors='pt', padding=True).to("cuda:0")

    outputs = model.generate(input_ids.input_ids,attention_mask=input_ids.attention_mask, max_new_tokens=500)

    generated_texts = []
    for i in range(len(prompts)):
        generated_ids = outputs[i, input_ids.input_ids.shape[1]:]
        generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        generated_texts.append(generated_text)

    return generated_texts

# test
print("\n\n".join(generate(["I get hit", "Who are you?"])))


# %%
!wget https://NLP-course-cuhksz.github.io/Assignments/Assignment1/task1/data/1.exam.json

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

RuntimeError: cublasLt ran into an error!
	shapeA=torch.Size([3584, 3584]), shapeB=torch.Size([66, 3584]), shapeC=(66, 3584)
	(lda, ldb, ldc)=(c_int(3584), c_int(3584), c_int(3584))
	(m, n, k)=(c_int(3584), c_int(66), c_int(3584))

In [11]:
# %%
import json

with open('1.exam.json') as f:
  data = json.load(f)
  data = data[:20] # just for demo

print(data[0])

# %%
your_prompt = """请回答下面的多选题，你需要先给出一段解答分析，然后再按照下面的格式样例给出正确答案，格式样例：[答案] B
{question}
{options}"""

def get_query(da):
  da['options'] = '\n'.join([f"{k}:{v}" for k, v in da['option'].items() if v])
  return your_prompt.format_map(da)

for item in data:
  item['query'] = get_query(item)


print(data[0]['query'])

# %%
model_answers = generate([item['query'] for item in data])
print(f'\n{model_answers[0]}')

# %%
import re
from tqdm import tqdm

def get_ans(ans):
    match = re.findall(r'.*?([A-E]+(?:[、, ]+[A-E]+)*)', ans)
    if match:
        last_match = match[-1]
        return ''.join(re.split(r'[、, ，]+', last_match))
    return ''

correct_num = 0
total_num = 0
for model_answer, item in tqdm(zip(model_answers, data)):
  if get_ans(model_answer) == item['answer']:
    correct_num += 1
  total_num += 1
  item['model_answer'] = model_answer

print(f"ACC: {correct_num/total_num:.2%}")

result_path = "/content/result.jso"
with open(result_path, "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)
    print(f"Results are save in {result_path}")

FileNotFoundError: [Errno 2] No such file or directory: '1.exam.json'